In [1]:
%run models/neural_nets.py


[1,    30] loss: 2.04228
example:  predicted tensor([[0.4798, 0.5202],
        [0.4821, 0.5179],
        [0.4937, 0.5063],
        [0.4547, 0.5453],
        [0.4481, 0.5519]]) , actual tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.]]) 

------------------------------
[2,    30] loss: 1.96896
example:  predicted tensor([[0.5452, 0.4548],
        [0.4842, 0.5158],
        [0.4761, 0.5239],
        [0.4833, 0.5167],
        [0.5355, 0.4645]]) , actual tensor([[1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.]]) 

------------------------------
[3,    30] loss: 1.88677
example:  predicted tensor([[0.4438, 0.5562],
        [0.4223, 0.5777],
        [0.4437, 0.5563],
        [0.5243, 0.4757],
        [0.4294, 0.5706]]) , actual tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.]]) 

------------------------------
[4,    30] loss: 1.76487
example:  predicted tensor([[0.3521, 0.6479],
  

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 1) Load data
df = pd.read_csv("mushroom_dataset.csv")

# 2) Extract raw X and y
X_raw = df.drop(columns="class")
y_raw = df["class"]

# 3) Label‑encode the target
le = LabelEncoder().fit(y_raw)
y = le.transform(y_raw)    # 0='e', 1='p'

# 4) One‑hot encode all features
X_oh_df = pd.get_dummies(X_raw, drop_first=False)
feature_names_oh = X_oh_df.columns.tolist()
X_oh = X_oh_df.values      # convert to numpy

# 5) Stratified train/test split
X_train_oh, X_test_oh, y_train, y_test = train_test_split(
    X_oh, y,
    test_size=0.30,
    random_state=42,
    stratify=y
)

# Quick sanity checks
print("One‑hot shape:", X_oh.shape)
print("Features:", len(feature_names_oh))
print("Train/test sizes:", X_train_oh.shape[0], X_test_oh.shape[0])



One‑hot shape: (8124, 117)
Features: 117
Train/test sizes: 5686 2438


In [4]:
from sklearn_extra.cluster import KMedoids

# 6) Find 40 medoids on the ONE‑HOT test set
kmed = KMedoids(n_clusters=40, metric="manhattan", random_state=0)
kmed.fit(X_test_oh)

# medoid indices into X_test_oh
med_idx    = kmed.medoid_indices_
# the actual medoid rows (one‑hot)
medoids_oh = X_test_oh[med_idx]

print(f"Selected {len(med_idx)} medoids from test set")
print("Example medoid indices:", med_idx[:10])
print("Medoids shape:", medoids_oh.shape)  # should be (40, n_features)


Selected 40 medoids from test set
Example medoid indices: [ 736  589 2233 1180    3 1549 1930 1764 2187  514]
Medoids shape: (40, 117)


In [13]:
import torch.nn.functional as F
import torch

def nn_proba_oh(arr_oh):
    """
    1) arr_oh: 1‑D or 2‑D numpy of one‑hot dummies
    2) convert each sample back to label‑encoded ints
    3) run through your NN (which expects integer codes + embeddings)
    4) return a (n_samples × 2) numpy array of softmax probs
    """
    arr = np.atleast_2d(arr_oh)
    # 1: convert one‑hot → label‑encoded vector
    X_le_batch = np.stack([onehot_to_label_vect(r) for r in arr])
    # 2: to torch.LongTensor
    t = torch.from_numpy(X_le_batch).long()
    # 3: forward pass + softmax
    with torch.no_grad():
        logits = model(t)
        probs  = F.softmax(logits, dim=1).cpu().numpy()
    return probs



In [7]:
from lime.lime_tabular import LimeTabularExplainer

explainer = LimeTabularExplainer(
    training_data         = X_train_oh,
    feature_names         = feature_names_oh,
    class_names           = ["edible","poisonous"],
    categorical_features  = list(range(X_train_oh.shape[1])),
    discretize_continuous = False,
    random_state          = 0
)


In [11]:
import numpy as np

# — you should already have these from the previous cell —
# X_raw            : DataFrame of original string‐valued features
# feature_names_oh : list of one‐hot column names (e.g. "odor_n", "odor_f", …)
# medoids_oh       : numpy array of shape (40, n_dummies)
# encoders         : dict mapping each original feature → its sklearn LabelEncoder
# 1) Rebuild the encoders for each original feature
encoders = {}
for feat in X_raw.columns:
    le = LabelEncoder().fit(X_raw[feat])
    encoders[feat] = le

# Quick check
print("Saved encoders for:", list(encoders.keys())[:5], "… total:", len(encoders))
# 1) Build a mapping from each raw feature to its dummy columns
orig_feats = X_raw.columns.tolist()   # same order you used for get_dummies
feat_to_cols = {
    feat: [col for col in feature_names_oh if col.startswith(feat + "_")]
    for feat in orig_feats
}

# 2) Conversion function
def onehot_to_label_vect(row_oh):
    """
    row_oh: 1-D numpy array of length == len(feature_names_oh)
    Returns: 1-D numpy int‐array of length == len(orig_feats)
    """
    x_le = []
    for feat in orig_feats:
        cols = feat_to_cols[feat]
        # grab that block of dummy values
        idxs  = [feature_names_oh.index(c) for c in cols]
        block = row_oh[idxs]
        # choose the dummy==1 (argmax over 0/1)
        k     = int(np.argmax(block))
        # parse category letter from the dummy name
        cat   = cols[k].split("_", 1)[1]
        # turn it back into the integer code
        code  = encoders[feat].transform([cat])[0]
        x_le.append(code)
    return np.array(x_le, dtype=np.int64)

# 3) Test it on the first medoid
test_row_le = onehot_to_label_vect(medoids_oh[0])
print("Label‐encoded vector for medoid #0:", test_row_le)

# 4) Sanity‐check: run it through your NN wrapper
proba = nn_proba_oh(medoids_oh[0])
print("NN predict_proba on medoid #0:", proba)


Saved encoders for: ['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor'] … total: 22
Label‐encoded vector for medoid #0: [2 3 3 1 5 1 0 0 7 1 1 2 2 7 7 0 2 1 4 2 4 0]
NN predict_proba on medoid #0: [[0.5230687  0.47693124]]


In [17]:
from collections import Counter, defaultdict
import numpy as np
from lime.lime_tabular import LimeTabularExplainer
import text_utils

agg_freq, agg_wt = Counter(), defaultdict(float)
seeds = [0,1,2]
d = X_train_oh.shape[1]
cat_feats = list(range(d))

for seed in seeds:
    explainer = LimeTabularExplainer(
        training_data         = X_train_oh,
        feature_names         = feature_names_oh,
        class_names           = ["edible","poisonous"],
        categorical_features  = cat_feats,
        discretize_continuous = False,
        random_state          = seed
    )
    for row in medoids_oh:
        # build the explanation (no label arg needed)
        exp = explainer.explain_instance(
            row,
            nn_proba_oh,
            num_features=d
        )

        # Grab the single key in exp.as_map(): the class LIME explained
        exp_map = exp.as_map()
        cls_key = next(iter(exp_map))    # either 0 or 1
        raw     = exp_map[cls_key]       # list of (feat_idx, weight)

        # filter only the first 5 active dummies:
        kept = 0
        for feat_idx, wt in raw:
            if row[feat_idx] == 1:
                feat_name     = feature_names_oh[feat_idx]
                agg_freq[feat_name] += 1
                agg_wt[feat_name]   += wt
                kept += 1
                if kept == 5:
                    break

# Build your summary_df as before...
total = len(medoids_oh)*len(seeds)
rows = [{
    "feature": feat,
    "pct_in_top5": 100*freq/total,
    "mean_signed_weight": agg_wt[feat]/freq
} for feat,freq in agg_freq.items()]

summary_df = (pd.DataFrame(rows)
                .sort_values(["pct_in_top5","mean_signed_weight"], ascending=False)
                .reset_index(drop=True))

print(summary_df.head(15))
text_utils.ensure_directory_exists("eval/lime_results")
summary_df.to_csv("eval/lime_results/lime_nn_summary.csv", index=False)


                       feature  pct_in_top5  mean_signed_weight
0   stalk-surface-above-ring_s    70.000000            0.011636
1                       odor_n    62.500000            0.038329
2     stalk-color-below-ring_w    46.666667            0.011111
3          spore-print-color_k    37.500000            0.022034
4                 population_v    33.333333           -0.010626
5          spore-print-color_n    30.000000            0.027900
6                 gill-color_n    25.000000            0.029302
7     stalk-color-above-ring_p    20.000000           -0.013371
8                       odor_f    20.000000           -0.064815
9                 gill-color_w    17.500000            0.015621
10               ring-number_o    17.500000           -0.010365
11  stalk-surface-below-ring_k    17.500000           -0.016537
12         spore-print-color_w    17.500000           -0.017658
13  stalk-surface-above-ring_k    17.500000           -0.040579
14                veil-color_w    16.666